In [1]:
import OpenOPC as OP
import pywintypes
pywintypes.datetime=pywintypes.TimeType #Se usa para OpenOPC o.o
import socket 

### Definicion de variables globales

In [2]:
opcserver = "ABB.AC800MC_OpcDaServer.3"
variables=[]
#Defino el objeto servidor OPC
opc=OP.client()

### Funciones


#funcion que lee el OPC
def actualizar():
    #Conecto con el servidor de ABB...
    opc.connect('ABB.AC800MC_OpcDaServer.3')
    #...y traigo una lista de variables
    vars=list(opc.read(["Applications.CAPEM.SC_TESTIGO.Value","Applications.CAPEM.SC_POS_Val.Value","Applications.CAPEM.SC_ERROR.Value",
                        "Applications.CAPEM.PT030_In.Value","Applications.CAPEM.LT035_mmH2O_GLOBAL.Value","Applications.CAPEM.TT024B_In.Value",
                        "Applications.CAPEM.TT024A_In.Value","Applications.CAPEM.TBCe_In.Value"]))
    #Agrego un miembro en la primera posicion de la lista, que despues se va a usar para chequeo de comunicacion
    #Reemplazo los TRUE y FALSE  por 1 y 0
    vars2= [0]+list(map((lambda x: 1 if x[1]==True else (0 if x[1]==False else x[1])),vars))
    return vars2

In [3]:
#funcion que lee el OPC
def actualizar():
    #Conecto con el servidor de ABB...
    opc.connect('ABB.AC800MC_OpcDaServer.3')
    #...y traigo una lista de variables
    vars=list(opc.read(["Applications.CAPEM.SC_TESTIGO","Applications.CAPEM.SC_POS_Val","Applications.CAPEM.SC_ERROR",
                        "Applications.CAPEM.PT030_In.Value","Applications.CAPEM.LT035_mmH2O_GLOBAL.Value","Applications.CAPEM.TT024B_In.Value",
                        "Applications.CAPEM.TT024A_In.Value","Applications.CAPEM.TBCe_In.Value"]))
    #Agrego un miembro en la primera posicion de la lista, que despues se va a usar para chequeo de comunicacion
    #Reemplazo los TRUE y FALSE  por 1 y 0
    vars2= [0]+list(map((lambda x: 1 if x[1]==True else (0 if x[1]==False else x[1])),vars))
    return vars2

### Rutina principal

In [4]:
port=9999
#Creo el objeto socket para comunicar con la otra PC
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    #Asocio el socket al puerto
    s.bind(('', port))
    ## Loop principal
    while True:
        #Con listen() habilito a escuchar conexiones y con accept() la acepto
        s.listen()
        conn, addr = s.accept() #devuelve conn que es como un nuevo socket exclusivo (creo)
        with conn:
            try:
                print(f"Connected by {addr}")
                #Mientras está la conexion se ejecuta el loop de envío de datos
                while True:
                    datos=actualizar()                      #Los traigo de OPC
                    data = conn.recv(1024).decode("ascii")  #Decodifico los datos enviados desde la otra PC
                    if not data or data=="cerrar":
                        break
                    #print(data)
                    if data=="hs":                          #Uso hs para realizar un handshake con la otra PC cuando se conecta (solo una vez)
                        resp=b"hs"
                    elif 0<=int(data)<=1000 and datos!=None:#Si lo que envía la PC es un numero es porque píde datos de OPC
                        cn=int(data)
                        datos[0]=cn 
                        #print(datos)                        #el dato que envía es un numero de confirmación, por lo que tengo que enviarlo en la pos 0
                        resp=str(datos).encode("ascii")     #Codifico los datos de OPC mas el cn

                    conn.sendall(resp)                      #y lo mando a la otra PC
            except Exception as e:
                print(str(e))

Connected by ('192.168.99.15', 52749)
Connected by ('192.168.99.15', 52750)
Connected by ('192.168.99.15', 52751)
[WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto
Connected by ('192.168.99.15', 52828)
[WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto
